Объединяю и чищу объявления WW2

In [1]:
# импорт библиотек
import pandas as pd
pd.options.mode.chained_assignment = None  # отключаю предупреждения default='warn'

import os


from transliterate import translit, get_available_language_codes

# прогресс-бар
from tqdm.notebook import trange, tqdm

### Объединяю файлы

In [2]:
# путь к файлам

file_path = 'C:/00_Data/temp_data/'

In [3]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'new/'):
    
    for filename in files:
        file_list += [filename]

In [17]:
# объединяю в один датафрейм

new_work = pd.DataFrame() # новый датафрейм

for file in tqdm(file_list):
    temp = pd.read_excel(file_path + 'new/' + file)
    
    temp.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)
    
    new_work = new_work.append(temp, ignore_index = True)
    
print('Количество строк: ', new_work.shape[0])

# удаляю дубликаты
new_work = new_work.drop_duplicates()

print('Количество строк после удаления дублей: ', new_work.shape[0])

  0%|          | 0/98 [00:00<?, ?it/s]

Количество строк:  176654
Количество строк после удаления дублей:  176654


In [29]:
# вытаскиваю названия файлов в список file_list

file_list = []

for root, dirs, files in os.walk(file_path + 'new/'):
    
    for filename in files:
        file_list += [filename]

In [37]:
# объединяю в один датафрейм


for file in tqdm(file_list[55:]):
    temp = pd.read_excel(file_path + 'new/' + file)
    print('Количество строк: ', temp.shape[0])
    
    temp = temp[~(temp.name.str.contains('чердачн', case=False))]
    temp = temp[~(temp.name.str.contains('складск', case=False))]

    print('Количество строк: ', temp.shape[0])
    
    temp.to_excel(file_path + 'new/' + file, index=False)

  0%|          | 0/41 [00:00<?, ?it/s]

Количество строк:  1742
Количество строк:  1734
Количество строк:  597
Количество строк:  596
Количество строк:  175
Количество строк:  175
Количество строк:  6344
Количество строк:  6329
Количество строк:  1502
Количество строк:  1501
Количество строк:  8321
Количество строк:  8312
Количество строк:  1342
Количество строк:  1340
Количество строк:  2479
Количество строк:  2474
Количество строк:  297
Количество строк:  297
Количество строк:  66
Количество строк:  66
Количество строк:  31
Количество строк:  31
Количество строк:  1028
Количество строк:  1025
Количество строк:  147
Количество строк:  146
Количество строк:  590
Количество строк:  590
Количество строк:  64
Количество строк:  64
Количество строк:  1868
Количество строк:  1868
Количество строк:  274
Количество строк:  274
Количество строк:  1356
Количество строк:  1356
Количество строк:  287
Количество строк:  287
Количество строк:  4423
Количество строк:  4423
Количество строк:  1495
Количество строк:  1495
Количество строк: 

In [18]:
new_work.drop_duplicates(subset=['topic_url'], keep='last', inplace=True)

In [19]:
new_work.shape

(176318, 24)

In [20]:
new_work.sample(2)

,topic_url,category,name,description,price,price_type,item_number,country,region,start_date,...,delivery,delivery_cost,payment,nickname,seller_status,seller_group,rate_pos,rate_neg,user_details,temp_dict
140552,https://forum.ww2.ru/index.php?showtopic=4769113,"СТ, Легковые автомобили, Архив","Виллис,рулевая сошка",Цена 500р.+почта,Не продано,блиц цена,none,Россия,Брянщина,25 ноя 2018,...,Почта России,none,Оплата на карту,*******,none,Член клуба форума,396.0,-1.0,*******\nЧлен клуба форума\n5 467 сообщений\nН...,"{'Страна продавца': 'Россия', 'Расположение ло..."
94681,https://forum.ww2.ru/index.php?showtopic=4853620,"ММГ, Боеприпасы, Архив",Гильзы SS,Обойма,Не продано,блиц цена,none,Россия,Schlisselburg,21 фев 2020,...,Почта России,300 ₽,Оплата на карту,*******,none,Пользователи,108.0,0.0,*******\nПользователи\n808 сообщений\nНа форум...,"{'Страна продавца': 'Россия', 'Расположение ло..."


### Добавляю статус продажи

In [21]:
# разделяю датасет на два

new_work_actual = new_work[~new_work.category.str.contains('Архив', case=True)]
new_work_archive = new_work[new_work.category.str.contains('Архив', case=True)]

In [22]:
new_work_archive.shape[0] + new_work_actual.shape[0] == new_work.shape[0]

True

In [23]:
# 2 -- для вещей, которые ещё в продаже

new_work_actual['sale_status'] = 2

In [24]:
# 1 -- для проданных вещей, 0 -- для непроданных

new_work_archive['sale_status'] = new_work_archive['price']

new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace('Снято с торгов администрацией', '0')
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace(r'Не продано.+', '0', regex=True)
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace(r'Продано.+', '1', regex=True)
new_work_archive['sale_status'] = new_work_archive['sale_status'].str.replace(r'\d.+', '1', regex=True)

# Удаляю Продано из цен
new_work_archive['price'] = new_work_archive['price'].str.replace('Продано, ', '')

In [25]:
new_work_archive.sale_status.value_counts()

Не продано    86325
1             58585
0               555
Name: sale_status, dtype: int64

In [26]:
# объединяю

new_work = pd.concat([new_work_archive, new_work_actual], axis=0)

In [27]:
# убираю Архив из категорий

new_work['category'] = new_work['category'].str.replace(', Архив', '')

In [16]:
new_work.price

0               1000
1               1000
2         Не продано
3                300
4                900
             ...    
176649          3000
176650          6000
176651         25000
176652          5200
176653          6000
Name: price, Length: 176318, dtype: object

### Убираю лишнее

In [15]:
new_work['name'] = new_work['name'].str.replace(r'[^\w\s]+', ' ', regex=True)
new_work['name'] = new_work['name'].str.replace(r'\s+', ' ', regex=True)
new_work['name'] = new_work['name'].str.replace('ё', 'e', regex=True)

In [16]:
new_work['name'].fillna('none', inplace=True)

In [17]:
new_work.shape

(176381, 25)

In [18]:
unless_things = ['шинел', 'бурка', 'куртка', 'гимнастер', 'рубаха', 'нательное', 'белье',
                 'брюк', 'шаровар', 'штаны', 'галиф', 'бридж', 'лампасы',
                 'сапоги', 'ботинки', 'гамаши',
                 'пилотка', 'кепка', 'фуражка', 'папаха',
                 'погоны', 'воротник', 'эполет',
                 'кубик', 'кубари', 'катушки', 'шпалы', 'треугольник', 'петлицы',
                 'сюртук', 'френч', 'мундир', 'китель', 'маскостюм', 'халат', 'платье',
                 'лента', 'повязка',
                 'США', 'ленд лиз', 
                 'афганка', 'форма', 'униформ', 'плащ-палатка']

In [19]:
%%time

for thing in unless_things:
    new_work = (new_work[~
                         (
                             (new_work.category.str.contains('униформа', case=False)) &
                             (new_work.name.str.contains(f'{thing}', case=False))
                         )])

Wall time: 13.1 s


In [20]:
new_work.shape

(174420, 25)

In [ ]:
(new_work['name'].value_counts().reset_index()
#.to_excel(file_path + 'temp.xlsx', index=False)
)

In [21]:
new_work['name'].value_counts().reset_index().to_excel(file_path + 'temp.xlsx', index=False)

In [ ]:
new_work.sample()

In [ ]:
(new_work.price.value_counts().reset_index()
 .to_excel(file_path + 'sample.xlsx', index=False)
)

### Чищу регионы

In [49]:
new_work['region_clear'] = new_work['region'].copy()

In [50]:
new_work['region_clear'] = new_work['region_clear'].str.lower()
new_work['region_clear'] = new_work['region_clear'].str.replace('г. ', '', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace(r'^г ', '', regex=True)

new_work['region_clear'] = new_work['region_clear'].str.replace('россия', '', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('рф', '', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('обл.', 'область', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('спб', 'санкт-петербург', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('c петербург', 'санкт-петербург', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('россия санкт петербург', 'санкт-петербург', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('с петербург лен область', 'санкт-петербург', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('санкт петербург', 'санкт-петербург', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('питер', 'санкт-петербург', regex=False)

new_work['region_clear'] = new_work['region_clear'].str.replace('мск', 'москва', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('ьрянск', 'брянск', regex=False)

new_work['region_clear'] = new_work['region_clear'].str.replace('astrachan', 'астрахань', regex=False)
new_work['region_clear'] = new_work['region_clear'].str.replace('chelyabinsk', 'челябинск', regex=False)

new_work['region_clear'] = new_work['region_clear'].str.replace(r'[^\w\s]+', ' ', regex=True)
new_work['region_clear'] = new_work['region_clear'].str.replace('\s+', ' ', regex=True )

new_work['region_clear'].fillna('none', inplace=True)

new_work['region_clear'] = new_work['region_clear'].apply(lambda x: translit(x, 'ru'))

In [51]:
new_work['region_clear'] = new_work['region_clear'].apply(lambda x: str(x).strip())

new_work['region_clear'].fillna('none', inplace=True)

In [52]:
new_work['region_clear'].nunique()

1270

In [56]:
#new_work['region_clear'].value_counts().reset_index().to_excel(file_path + 'region.xlsx', index=False)

In [69]:
clear_regions = pd.read_excel('C:/00_Data/temp_data/WW2/clear_regions.xlsx', sheet_name='data_need')

In [71]:
new_work = new_work.merge(clear_regions, how='left', on='region_clear')

In [72]:
new_work.drop(columns=['region_clear'], inplace=True)

In [75]:
new_work.region_new.value_counts().reset_index().head(10)

,index,region_new
0,Волгоградская область,21270
1,Калининградская область,20458
2,Москва,17468
3,Смоленская область,15687
4,Санкт-Петербург,14267
5,Новгородская область,8540
6,Брянская область,6781
7,Московская область,5241
8,Псковская область,4455
9,Тверская область,4397


In [77]:
new_work.columns

Index(['topic_url', 'category', 'name', 'description', 'price', 'price_type',
       'item_number', 'country', 'region', 'start_date', 'update_date',
       'views', 'authenticity', 'watching', 'delivery', 'delivery_cost',
       'payment', 'nickname', 'seller_status', 'seller_group', 'rate_pos',
       'rate_neg', 'user_details', 'temp_dict', 'sale_status', 'city_clear',
       'region_new'],
      dtype='object')

In [79]:
new_work.category.value_counts()

СТ, Грузовые автомобили                                    19800
ВА, Германия, Копаные                                      16608
СТ, Танки, бронетехника                                    16298
ММГ, Аксессуары, Германия                                  12049
ВА, Германия, Erkennungsmarke                              10585
ММГ, Боеприпасы                                             9641
ВА, Германия, Снаряжение, личные вещи                       9478
ММГ, Аксессуары, РИ, СССР, РФ                               7862
СТ, Мотоциклы, велосипеды, мотороллеры                      6871
ВА, Германия, Головные уборы, шлемы                         6774
СТ, Легковые автомобили                                     6062
СТ, Аксессуары                                              5914
СТ, Свалка                                                  4824
СТ, Артиллерия, прицепы, кухни                              3563
ВА, Германия, Ремни и пряги                                 3272
ВА, СССР 1945-1954, Снаря

In [81]:
new_work.sale_status.value_counts()

0    85821
1    58042
2    30495
Name: sale_status, dtype: int64

In [ ]:
new

In [89]:
new_work[new_work.sale_status == '1'].price.value_counts().reset_index()
#.to_excel(file_path + 'prices.xlsx')

,index,price
0,1000,6152
1,500,3993
2,2000,3322
3,1500,3265
4,3000,2492
...,...,...
2124,12700,1
2125,4644,1
2126,7467,1
2127,6990,1


In [92]:
new_work['sale_status'] = new_work['sale_status'].str.replace(r'Продано, ', '1', regex=True)

In [95]:
new_work = new_work.astype({'price': 'int32'})

ValueError: invalid literal for int() with base 10: 'Не продано'

In [94]:
new_work.dtypes

topic_url         object
category          object
name              object
description       object
price             object
price_type        object
item_number       object
country           object
region            object
start_date        object
update_date       object
views             object
authenticity     float64
watching         float64
delivery          object
delivery_cost     object
payment           object
nickname          object
seller_status     object
seller_group      object
rate_pos         float64
rate_neg         float64
user_details      object
temp_dict         object
sale_status       object
city_clear        object
region_new        object
dtype: object

In [93]:
new_work[new_work.sale_status == '1'].price.mean()

TypeError: Could not convert 10001000300900600020001500141751050347312300210030001389213900138923004000270060025009975152259002600152011002205220523151200100015755000550110020000155068251150800650150110025002500150018002300270010001000100028352000619512773000104915751001157545151470262526255000138921279206620112011280915225650600012601000100073505250273026254785220547253000220544681000030003255150015001500364755035115175021001000014700367526783000514520004100750100010001000100010001000100010001000100015003600250052501575110257757163312000300025003000150010002000400050010001000100010001000100010001000500130025001365500105160003000100080001500018002850300100080001500525010501200110812500888815750315073501500700200023000250010005000850530002625650120016801890120025002000105002000100010001300160030001656250018521500150015004200156852502000350070015501000200010506003473840051110322053505509501575178502550100055042001000500043001575136595015006000250015006000800700100020002000110015001785020